In [ ]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.constants import START
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command


class State(TypedDict):
    """The graph state."""

    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""


def node(state: State):
    answer = interrupt(
        # This value will be sent to the client
        # as part of the interrupt information.
        "what is your age?"
    )
    print(f"> Received an input from the interrupt: {answer}")
    return {"human_value": answer}


builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

# A checkpointer must be enabled for interrupts to work!
checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)

# > {'__interrupt__': (Interrupt(value='what is your age?', id='45fda8478b2ef754419799e10992af06'),)}

human_comment = input("input from user: ")
command = Command(resume=human_comment.strip())

for chunk in graph.stream(command, config):
    print(chunk)




> Received an input from the interrupt: hello
{'node': {'human_value': 'hello'}}


In [ ]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.constants import START
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command


class State(TypedDict):
    foo: str
    human_value: Optional[str]


def node(state: State):
    answer = interrupt("what is your age?")
    print(f"> Received an input from the interrupt: {answer}")
    return {"human_value": answer}


builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {"thread_id": str(uuid.uuid4())}
}

# --- Run lần đầu để gặp interrupt ---
for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)

# Lúc này graph sẽ trả ra interrupt
# Bạn hỏi input user:
user_input = input(">> Please enter your answer: ")

# --- Resume với input từ user ---
for chunk in graph.stream(Command(resume=user_input), config):
    print(chunk)


{'__interrupt__': (Interrupt(value='what is your age?', id='e265fbb647eb378df97f1b67e1767b38'),)}
> Received an input from the interrupt: hello
{'node': {'human_value': 'hello'}}
